In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

sns.set()

# Load data set
df = pd.read_csv("sgv.csv", sep=";")

# Verify shape is what it should be (cols, rows)
df.shape

# Display first 5 rows
df.head(n=5)

# Date defined in 3 seperarate cols -> merge into one
df['Date'] = df['Month'].astype(str) + '-' + df['Day_in_Month'].astype(str) + '-' + df['Year'].astype(str)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df.head()

# Print data type of each column
df.dtypes

# Encode Month and Day as categorical
df.Month = df.Month.astype('category')
df.Day = df.Day.astype('category')
df.dtypes

# Check for duplicates -> returns true or false
df.duplicated().any()
# if there were duplicates, display like this:
df[df.duplicated(keep=False)].head(n=10)
# and drop duplicates like this:
df.drop_duplicates(inplace=True)

# Check if there are NULL values
df.isna().any()
# or do `df.isna().any().any()` to get a true or false
df.isna().any().any()
# Replace null values with the mean value for each columns
df = df.fillna(df.mean())

# Examine minimum and maximum values and more stats (mean, std, quartils, count):
df.describe()
# update statements where some maximum seems wrong to a correct maximum (sunshine_percentage > 100 to 100)
df["Sunshine_Percentage"].values[df["Sunshine_Percentage"] > 100] = 100

# Plot the data
fig, ax = plt.subplots(figsize=(15,5))
df.Passengers.plot(title="Passengers over time", ax=ax)

# Correlation analysis
plt.subplots(figsize=(12, 12))
sns.heatmap(df.corr(), annot=True, cmap='RdYlGn_r', linewidths=0.5, fmt='.2f')

# Boxplot for every numerical value to take a look at outliers
numerical_cols = ['Passengers', 'Temperature_Midday', 'Snow_5Days', 'Wind', 'Temperature_Deviation',
                  'Temperature_Max', 'Temperature_Min', 'Temperature_Evening', 'Precipiation', 'Precipiation_5Days']
_ = df.loc[:, numerical_cols].plot(kind='box', subplots=True, layout=(5, 2), figsize=(10, 18), sharex=False)

# Amount of passengers during revisions
df[(df.Revision == 1) & (df.Passengers > 0)].head()
# Fix the error, set the passengers to 0
df.Passengers.values[df.Revision==1] = 0

# Compare min max values to check whetever its plausible
(df["Temperature_Min"] < df["Temperature_Max"]).all()

# Matrix showing snow days per month per year
pd.pivot_table(df, index=[df.Date.dt.month, "Month"], columns=["Year"], values=["Snow_5Days"])
# Fix the error we created by recplacing null values with mean
df.Snow_5Days.values[df.Month.isin(["June", "July", "August", "September", "October"])] = 0

# Automatic Data Quality Assessment with Pandas Profiling
import pandas_profiling
df2 = pd.read_csv("sgv.csv", sep=";")
profile = df2.profile_report(html={'style':{'full_width':True}})
# Save report uncomment line below, if you want to save the report
#profile.to_file(output_file="sgv-profile.html")
profile

# How many passengers in january 2006?
jan_2006 = df[(df['Date'].dt.year == 2006) & (df['Date'].dt.month == 1)]
total_passengers = jan_2006['Passengers'].sum()
print(f"Total passengers in January 2006: {total_passengers}")

# Correlation of passengers and revisions
correlation = df['Passengers'].corr(df['Revision'])
print(f"Korrelation: {correlation}")
